<a href="https://colab.research.google.com/github/Us9r/DicodingBMLP/blob/main/%5BClustering%5D_Submission_Akhir_BMLP_Us9r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset **tanpa label** dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.
   
2. **Ketentuan Dataset**:
   - **Tanpa label**: Dataset tidak boleh memiliki label atau kelas.
   - **Jumlah Baris**: Minimal 1000 baris untuk memastikan dataset cukup besar untuk analisis yang bermakna.
   - **Tipe Data**: Harus mengandung data **kategorikal** dan **numerikal**.
     - *Kategorikal*: Misalnya jenis kelamin, kategori produk.
     - *Numerikal*: Misalnya usia, pendapatan, harga.

3. **Pembatasan**:  
   Dataset yang sudah digunakan dalam latihan clustering (seperti customer segmentation) tidak boleh digunakan.

## Pengenalan Dataset Diets Nutrition


Dataset **Diets Nutrition** berisi informasi nutrisi dari **7806 resep** yang mencakup berbagai jenis diet (seperti **paleo**, **keto**, **vegan**) dan masakan (seperti **Amerika**, **Meksiko**, **Asia**). Dataset ini mencakup nilai **protein**, **karbohidrat**, dan **lemak**, yang berguna untuk analisis data dan pembelajaran mesin terkait makanan dan kesehatan.

### Sumber Dataset

Dataset ini diambil dari **Kaggle**, platform data terbuka yang menyediakan berbagai dataset untuk analisis, penelitian, dan kompetisi. Dataset ini diunggah oleh pengguna bernama **"onydrive".**<sup>[[1]]*credit belong to respective owner.*</sup>

### Ketentuan Dataset

- **Tanpa Label**: Dataset tidak memiliki label atau kelas.
- **Jumlah Baris**: Terdiri dari **7806 baris**.
- **Tipe Data**: Mengandung data:
  - **Kategorikal**: (seperti `Diet_type` dan `Cuisine_type`)
  - **Numerikal**: (seperti `Protein(g)`, `Carbs(g)`, dan `Fat(g)`)

[1]: https://www.kaggle.com/datasets/onydrive/diets-nutrition-dataset

# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [20]:
import pandas as pd

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

In [21]:
df = pd.read_csv("https://raw.githubusercontent.com/Us9r/vault/main/DicodingBMLP/All_Diets.csv")
df

,Diet_type,Recipe_name,Cuisine_type,Protein(g),Carbs(g),Fat(g),Extraction_day,Extraction_time
0,paleo,Bone Broth From 'Nom Nom Paleo',american,5.22,1.29,3.20,2022-10-16,17:20:09
1,paleo,"Paleo Effect Asian-Glazed Pork Sides, A Sweet ...",south east asian,181.55,28.62,146.14,2022-10-16,17:20:09
2,paleo,Paleo Pumpkin Pie,american,30.91,302.59,96.76,2022-10-16,17:20:09
3,paleo,Strawberry Guacamole recipes,mexican,9.62,75.78,59.89,2022-10-16,17:20:09
4,paleo,"Asian Cauliflower Fried ""Rice"" From 'Nom Nom P...",chinese,39.84,54.08,71.55,2022-10-16,17:20:09
...,...,...,...,...,...,...,...,...
7801,dash,Brown Butter-Sunchoke Soup With Brussels Sprou...,italian,85.20,288.14,137.15,2022-10-16,20:40:44
7802,dash,Make-Your-Own-Salad,american,141.98,123.18,323.50,2022-10-16,20:40:44
7803,dash,Luis Buñuel Dry Martini,world,0.01,0.39,0.00,2022-10-16,20:40:44
7804,dash,Cornflake Semi-Fried Chicken Tenders,american,155.38,239.88,260.84,2022-10-16,20:40:44


# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset. EDA bertujuan untuk:

1. **Memahami Struktur Data**
   - Tinjau jumlah baris dan kolom dalam dataset.  
   - Tinjau jenis data di setiap kolom (numerikal atau kategorikal).

2. **Menangani Data yang Hilang**  
   - Identifikasi dan analisis data yang hilang (*missing values*). Tentukan langkah-langkah yang diperlukan untuk menangani data yang hilang, seperti pengisian atau penghapusan data tersebut.

3. **Analisis Distribusi dan Korelasi**  
   - Analisis distribusi variabel numerik dengan statistik deskriptif dan visualisasi seperti histogram atau boxplot.  
   - Periksa hubungan antara variabel menggunakan matriks korelasi atau scatter plot.

4. **Visualisasi Data**  
   - Buat visualisasi dasar seperti grafik distribusi dan diagram batang untuk variabel kategorikal.  
   - Gunakan heatmap atau pairplot untuk menganalisis korelasi antar variabel.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7806 entries, 0 to 7805
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Diet_type        7806 non-null   object 
 1   Recipe_name      7806 non-null   object 
 2   Cuisine_type     7806 non-null   object 
 3   Protein(g)       7806 non-null   float64
 4   Carbs(g)         7806 non-null   float64
 5   Fat(g)           7806 non-null   float64
 6   Extraction_day   7806 non-null   object 
 7   Extraction_time  7806 non-null   object 
dtypes: float64(3), object(5)
memory usage: 488.0+ KB


In [35]:
df.describe()

,Protein(g),Carbs(g),Fat(g)
count,7806.000000,7806.000000,7806.000000
mean,83.231498,152.123189,117.328542
std,89.797282,185.907322,122.098117
min,0.000000,0.060000,0.000000
25%,24.415000,36.162500,41.067500
50%,56.280000,93.415000,84.865000
75%,112.357500,205.915000,158.290000
max,1273.610000,3405.550000,1930.240000


In [23]:
df.isnull().sum()

,0
Diet_type,0
Recipe_name,0
Cuisine_type,0
Protein(g),0
Carbs(g),0
Fat(g),0
Extraction_day,0
Extraction_time,0


# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning. Data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang perlu dilakukan, namun **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

In [24]:
duplicate_rows = df[df.duplicated()]
display(duplicate_rows)

,Diet_type,Recipe_name,Cuisine_type,Protein(g),Carbs(g),Fat(g),Extraction_day,Extraction_time
2778,vegan,Strawberry Rhubarb Jam {Vegan},american,2.48,155.82,22.73,2022-10-16,20:13:34
2789,vegan,Strawberry Rhubarb Jam {Vegan},american,2.48,155.82,22.73,2022-10-16,20:13:40
4338,mediterranean,Mediterranean Marinade,mediterranean,1.50,12.79,68.75,2022-10-16,16:43:20
4412,mediterranean,Mediterranean Grain Salad,mediterranean,14.02,48.13,16.24,2022-10-16,16:43:47
5184,mediterranean,Mediterranean Dip,mediterranean,21.71,40.46,32.12,2022-10-16,16:48:18


In [25]:
df = df.drop(columns=['Recipe_name','Extraction_day', 'Extraction_time'])

df

,Diet_type,Cuisine_type,Protein(g),Carbs(g),Fat(g)
0,paleo,american,5.22,1.29,3.20
1,paleo,south east asian,181.55,28.62,146.14
2,paleo,american,30.91,302.59,96.76
3,paleo,mexican,9.62,75.78,59.89
4,paleo,chinese,39.84,54.08,71.55
...,...,...,...,...,...
7801,dash,italian,85.20,288.14,137.15
7802,dash,american,141.98,123.18,323.50
7803,dash,world,0.01,0.39,0.00
7804,dash,american,155.38,239.88,260.84


In [26]:
for column in df.select_dtypes(include=['object']).columns: # pilih kolom bertipe 'object'
       print(f"\nColumn: {column}")
       print("Unique Values:", df[column].unique())
       print("Value Counts:\n", df[column].value_counts())


Column: Diet_type
Unique Values: ['paleo' 'vegan' 'keto' 'mediterranean' 'dash']
Value Counts:
 Diet_type
mediterranean    1753
dash             1745
vegan            1522
keto             1512
paleo            1274
Name: count, dtype: int64

Column: Cuisine_type
Unique Values: ['american' 'south east asian' 'mexican' 'chinese' 'mediterranean'
 'italian' 'french' 'indian' 'nordic' 'eastern europe' 'central europe'
 'kosher' 'british' 'caribbean' 'south american' 'middle eastern' 'asian'
 'japanese' 'world']
Value Counts:
 Cuisine_type
american            2907
mediterranean       1744
italian              799
french               604
world                259
british              239
mexican              224
nordic               152
south east asian     148
south american       137
asian                126
chinese              120
indian                92
middle eastern        91
eastern europe        55
japanese              50
central europe        34
caribbean             18
kosher  

In [27]:
def group_cuisine(cuisine):
    if cuisine in ["american", "british", "mexican", "south american"]:
        return "Amerika"
    elif cuisine in ["mediterranean", "italian", "french", "nordic", "eastern europe", "central europe"]:
        return "Eropa"
    elif cuisine in ["asian","south east asian", "chinese", "indian", "japanese", "middle eastern"]:
        return "Asia"
    elif cuisine in ["caribbean", "kosher", "world"]:
        return "World"
    else:
        return "Other"  # Handling untuk cuisine yang tidak terdefinisi

df['Cuisine_Group'] = df['Cuisine_type'].str.lower().apply(group_cuisine)
df = df.drop(columns=['Cuisine_type'])

In [28]:
for column in df.select_dtypes(include=['object']).columns: # pilih kolom bertipe 'object'
       print(f"\nColumn: {column}")
       print("Unique Values:", df[column].unique())
       print("Value Counts:\n", df[column].value_counts())


Column: Diet_type
Unique Values: ['paleo' 'vegan' 'keto' 'mediterranean' 'dash']
Value Counts:
 Diet_type
mediterranean    1753
dash             1745
vegan            1522
keto             1512
paleo            1274
Name: count, dtype: int64

Column: Cuisine_Group
Unique Values: ['Amerika' 'Asia' 'Eropa' 'World']
Value Counts:
 Cuisine_Group
Amerika    3507
Eropa      3388
Asia        627
World       284
Name: count, dtype: int64


In [34]:
df

,Diet_type,Protein(g),Carbs(g),Fat(g),Cuisine_Group
0,paleo,5.22,1.29,3.20,Amerika
1,paleo,181.55,28.62,146.14,Asia
2,paleo,30.91,302.59,96.76,Amerika
3,paleo,9.62,75.78,59.89,Amerika
4,paleo,39.84,54.08,71.55,Asia
...,...,...,...,...,...
7801,dash,85.20,288.14,137.15,Eropa
7802,dash,141.98,123.18,323.50,Amerika
7803,dash,0.01,0.39,0.00,World
7804,dash,155.38,239.88,260.84,Amerika


# **6. Pembangunan Model Clustering**

## **a. Pembangunan Model Clustering**

Pada tahap ini, Anda membangun model clustering dengan memilih algoritma yang sesuai untuk mengelompokkan data berdasarkan kesamaan. Berikut adalah **rekomendasi** tahapannya.
1. Pilih algoritma clustering yang sesuai.
2. Latih model dengan data menggunakan algoritma tersebut.

In [29]:
#Type your code here

## **b. Evaluasi Model Clustering**

Untuk menentukan jumlah cluster yang optimal dalam model clustering, Anda dapat menggunakan metode Elbow atau Silhouette Score.

Metode ini membantu kita menemukan jumlah cluster yang memberikan pemisahan terbaik antar kelompok data, sehingga model yang dibangun dapat lebih efektif. Berikut adalah **rekomendasi** tahapannya.
1. Gunakan Silhouette Score dan Elbow Method untuk menentukan jumlah cluster optimal.
2. Hitung Silhouette Score sebagai ukuran kualitas cluster.

In [30]:
#Type your code here

## **c. Feature Selection (Opsional)**

Silakan lakukan feature selection jika Anda membutuhkan optimasi model clustering. Jika Anda menerapkan proses ini, silakan lakukan pemodelan dan evaluasi kembali menggunakan kolom-kolom hasil feature selection. Terakhir, bandingkan hasil performa model sebelum dan sesudah menerapkan feature selection.

In [31]:
#Type your code here

## **d. Visualisasi Hasil Clustering**

Setelah model clustering dilatih dan jumlah cluster optimal ditentukan, langkah selanjutnya adalah menampilkan hasil clustering melalui visualisasi.

Berikut adalah **rekomendasi** tahapannya.
1. Tampilkan hasil clustering dalam bentuk visualisasi, seperti grafik scatter plot atau 2D PCA projection.

In [32]:
#Type your code here

## **e. Analisis dan Interpretasi Hasil Cluster**

Setelah melakukan clustering, langkah selanjutnya adalah menganalisis karakteristik dari masing-masing cluster berdasarkan fitur yang tersedia.

Berikut adalah **rekomendasi** tahapannya.
1. Analisis karakteristik tiap cluster berdasarkan fitur yang tersedia (misalnya, distribusi nilai dalam cluster).
2. Berikan interpretasi: Apakah hasil clustering sesuai dengan ekspektasi dan logika bisnis? Apakah ada pola tertentu yang bisa dimanfaatkan?

In [33]:
#Type your code here

Tulis hasil interpretasinya di sini.
1. Cluster 1:
2. Cluster 2:
3. Cluster 3:

# **7. Mengeksport Data**

Simpan hasilnya ke dalam file CSV.